In [43]:
header_paths = """C:\Program Files (x86)\Vector CANape 13\CANapeAPI\CANapAPI.h
C:\Program Files (x86)\Vector CANape 14\CANapeAPI\CANapAPI.h
C:\Program Files (x86)\Vector CANape 15\CANapeAPI\CANapAPI.h
C:\Program Files (x86)\Vector CANape 16\CANapeAPI\CANapAPI.h
C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI.h"""

In [44]:
CANapAPI_dlls="""C:\Program Files (x86)\Vector CANape 13\CANapeAPI\CANapAPI64.dll
C:\Program Files (x86)\Vector CANape 14\CANapeAPI\CANapAPI64.dll
C:\Program Files (x86)\Vector CANape 15\CANapeAPI\CANapAPI64.dll
C:\Program Files (x86)\Vector CANape 16\CANapeAPI\CANapAPI64.dll
C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI64.dll"""

In [45]:
import re

In [46]:
fcn = re.compile("^extern ([^ ]+) ASAP3_EXPORT CALL_CONV ([^(]+)\(([^)]+)\);", flags=re.MULTILINE)

In [9]:
import os

In [10]:
for header_path in header_paths.split("\n")[::-1]:
    if os.path.exists(header_path):
        break

In [11]:
with open(header_path, "r") as fid:
    header = fid.read()

In [42]:
for f in fcn.findall(header):
    return_type, function_name, function_args = f
    
    print(function_name)
    
    #print(f[-1])
    if "\n" in f[-1]:
        pass
        #break

Asap3GetVersion
Asap3SetTCPOptions
Asap3Init
Asap3Init2
Asap3Init3
Asap3Init4
Asap3Init5
Asap3Init6
Asap3GetProjectDirectory
Asap3Exit
Asap3SetApplicationName
Asap3GetApplicationName
Asap3GetApplicationVersion
Asap3Exit2
Asap3ErrorText
Asap3PopupDebugWindow
Asap3SaveDebugWindow
Asap3AttachAsap2
Asap3CreateModule
Asap3CreateModule2
Asap3CreateModule3
Asap3CreateModule4
Asap3CreateModuleSec
Asap3GetModuleSecJobName
Asap3GetModuleCount
Asap3RestartMeasurementOnError
Asap3IsRestartMeasurementOnErrorEnabled
Asap3IsModuleActive
Asap3ModuleActivation
Asap3SwitchToMemoryPage
Asap3GetMemoryPage
Asap3GetDBObjectUnit
Asap3GetDBObjectInfo
Asap3GetDatabaseObjects
Asap3GetDatabaseObjectsByType
Asap3GetAsap2
Asap3TransmitFile2ClientPc
Asap3GetModuleName
Asap3GetModuleHandle
Asap3ReleaseModule
Asap3GetCommunicationType
Asap3ReadByAddress
Asap3WriteByAddress
Asap3ReadCalibrationObject
Asap3ReadCalibrationObject2
Asap3ReadCalibrationObjectEx
Asap3WriteCalibrationObject
Asap3WriteCalibrationObjectEx
Asap

In [38]:
return_type, function_name, function_args = f

In [14]:
args=f[-1]

In [24]:
args.split(",")

['TAsap3Hdl * hdl',
 '\n                                             unsigned long responseTimeout',
 '\n                                             const char *workingDir',
 '\n                                             unsigned long fifoSize',
 '\n                                             bool debugMode']

In [48]:
import jinja2
fcn_template = jinja2.Template("""
def {{ function_name }} 


""")

TemplateSyntaxError: Expected an expression, got 'end of print statement'

In [50]:
ctypes_template = jinja2.Template("""dll.Asap3Init5.restype = ctypes.c_bool
dll.Asap3Init5.argtypes = (
    ctypes.POINTER(TAsap3Hdl),
    ctypes.c_ulong,
    ctypes.c_char_p,
    ctypes.c_ulong,
    ctypes.c_ulong,
    ctypes.c_bool,
    ctypes.c_bool,
    ctypes.c_bool,
    ctypes.c_bool
)""")

In [70]:
lookup = {
    "unsigned long": "ctypes.c_ulong",
    "bool": "ctypes.c_bool",    
}

In [89]:
def to_c_type(arg_type):
    if arg_type in ["TAsap3Hdl", ]:
        return arg_type
    if arg_type in lookup:
        return lookup[arg_type]
    raise Exception(arg_type)

In [90]:
"TAsap3Hdl" in ["TAsap3Hdl", ]

True

In [93]:
print(f"{function_name}.argtypes=(")
for arg in args.split(","):
    arg_words = arg.strip().split(" ")
    var_name = arg_words[-1]
    arg_type = " ".join(arg_words[:-1])
    pointer = False
    if arg_type.endswith("*"):
        arg_type=arg_type.rstrip("*").strip()
        pointer=True
    elif var_name.startswith("*"):
        var_name=var_name.lstrip("*").strip()
        pointer=True
    if arg_type == "const char":
        c_type = "ctypes.c_char_p"
    else:
        c_type = to_c_type(arg_type)
    if pointer:
        c_type=f"ctypes.POINTER({c_type})"
    print(f"""    {c_type},""")
print(")")

Asap3OpenDisplay.argtypes=(
    ctypes.POINTER(TAsap3Hdl),
    ctypes.c_ulong,
    ctypes.POINTER(ctypes.c_char_p),
    ctypes.c_ulong,
    ctypes.c_bool,
)


In [84]:
c_type = to_c_type(return_type)

In [86]:
print(f"{function_name}.restype={c_type}")

Asap3OpenDisplay.restype=ctypes.c_bool
